## Handling Data

- Pytorch uses 2 primitives to work with data: either you provide a `torch.utils.data.DataLoader`, or `torch.utils.data.Dataset`

- What is the difference between `Dataloader` and `Dataset`? 
    - A `Dataset` simply stores samples and their labels
    - A `Dataloader` wraps an Iterable around a `Dataset`, this lets you conveniently yield batches from your data

- Note that Pytorch is subdivided in to domain-specific libraries (TorchText, TorchVision, and TorchAudio), and some things (like particularly datasets) should be retrieved from these modules

- Let's see an example of `Dataset` / `DataLoader` at work using FashionMNIST dataset

- Note that all `Dataset` objects includes 2 arguments; `transform` and `target_transform`
    - This lets you transform any data you wish to load, and can take in anything from `torchvision.transforms`
    - For example, if you want to rotate every picture in the dataset, you can pass in `RandomRotation`

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=False,
    transform=ToTensor()
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=False,
    transform=ToTensor(),
)

training_data.data.shape, training_data.targets.shape, test_data.data.shape, test_data.targets.shape

(torch.Size([60000, 28, 28]),
 torch.Size([60000]),
 torch.Size([10000, 28, 28]),
 torch.Size([10000]))

- After downloading the necessary data as a `Dataset`, we can wrap it into a `DataLoader`, which will conveniently help us batch the data later

In [3]:
batch_size = 64

train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models

- Any neural network in Pytorch can be defined as a class inheriting from `nn.Module`

- Define the layers in `__init__`, and specify how the data passes through the network by defining a method called `forward()`

- We can also speed up training by moving the operations to accelerators like CUDA, MPS, etc

In [5]:
## Check for accelerator
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [7]:
# Define model
class SomeNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x) ##each image is 28x28; flatten will create a 1x784 tensor
        logits = self.linear_relu_stack(x) ## 1x784 --> 512 --> 512 --> 10
        return logits
    
model = SomeNNModel().to(device)
print(model)

SomeNNModel(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Define training

- Obviously for training, we need a loss function and an optimiser

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

- Within a single loop, model does some prediction on a training dataset, and backpropagates the errors to the weights of the defined network
- Note that we do batch gradient descent, so errors are propagated by batch

In [33]:
def train(dataloader: DataLoader, model: nn.Module, loss_fn: nn.modules.loss._Loss, optimizer: torch.optim.Optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader: DataLoader, model: nn.Module, loss_fn: nn.modules.loss._Loss):
    size: int = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [36]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.649760  [   64/60000]
loss: 0.757617  [ 6464/60000]
loss: 0.517022  [12864/60000]
loss: 0.755429  [19264/60000]
loss: 0.669920  [25664/60000]
loss: 0.648152  [32064/60000]
loss: 0.728346  [38464/60000]
loss: 0.720863  [44864/60000]
loss: 0.719843  [51264/60000]
loss: 0.686887  [57664/60000]
Test Error: 
 Accuracy: 76.2%, Avg loss: 0.676699 

Epoch 2
-------------------------------
loss: 0.629107  [   64/60000]
loss: 0.737519  [ 6464/60000]
loss: 0.500381  [12864/60000]
loss: 0.742416  [19264/60000]
loss: 0.658399  [25664/60000]
loss: 0.636207  [32064/60000]
loss: 0.710728  [38464/60000]
loss: 0.710921  [44864/60000]
loss: 0.707912  [51264/60000]
loss: 0.673307  [57664/60000]
Test Error: 
 Accuracy: 76.8%, Avg loss: 0.662816 

Epoch 3
-------------------------------
loss: 0.610443  [   64/60000]
loss: 0.718951  [ 6464/60000]
loss: 0.485405  [12864/60000]
loss: 0.730503  [19264/60000]
loss: 0.648056  [25664/60000]
loss: 0.625643  [32064/600

## Save, Load, and Apply Models

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = SomeNNModel().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')